In [38]:
import dataset
import importlib
import json
import numpy as np
import os
import reasoning
import torch
import qa
import time

from dataset.bgquiz import BGQuiz
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForQuestionAnswering, BertForMultipleChoice
from reasoning.multichoice import *
from tqdm import tqdm
from qa.utils import query_es
from qa.wiki_abstracts import get_wiki_abstracts


# Reload the cached modules
_ = importlib.reload(dataset.bgquiz)
_ = importlib.reload(qa.utils)
_ = importlib.reload(reasoning.multichoice)
_ = importlib.reload(qa.wiki_abstracts)

In [5]:
model_dir = "models/RACE_BERT3"

# Example for a Bert model
model = BertForMultipleChoice.from_pretrained(model_dir, num_choices=4)
model.to('cuda')

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.0)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.0)
            )
          )
          (intermediate): Ber

In [6]:
tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=False)  # Add specific options if needed

In [7]:
wiki_control = \
'''
Обсадата на Одрин приключва с превземането на турската крепост от Втора българска армия под общото командване на ген. Никола Иванов. Решителният пробив на източния сектор на крепостта е извършен под командването на ген. Георги Вазов. Ръководещият защитата на Одрин – Шукри паша, предава сабята си на ген. Иванов с думите: „Храбростта на българската армия е безподобна. На такава храброст никоя крепост не може да устои“. Победата фактически решава изхода на Балканската война.
'''

question_control = "Кои военачалници ръководят защитата и превземането на Одринската крепост през месец март 1913 г.?"
options_control = [
         "Абдулах паша – ген. Радко Димитриев",
         "Явер паша – ген. Васил Кутинчев",
         "Назим паша – ген. Данаил Николаев",
         "Шукри паша – ген. Никола Иванов"
      ]

question_control = \
'''
Името на коя международна организация е пропуснато в текста от нейния Устав, 
подписан през 1945 г. в Сан Франциско?    
„1. _____________________________ е основана върху принципа на пълното равенство на  своите членове. 
3. Те се задължават да уреждат международните си спорове с мирни  средства. 
4. Членовете се въздържат в отношенията си от заплаха или от употреба на  сила. 
7. Организацията не се намесва във вътрешните работи на държавите. 
Това обаче не  засяга прилагането на принудителни мерки в случай на заплаха срещу мира, 
на нарушение на  мира и на актове на агресия.”'''
wiki_control = '''
Никола Иванов (генерал)\n\nНикола Иванов Иванов е български офицер (генерал от пехотата), началник на Щаба на войската през 1894–1896 година и министър на войната (1896–1899). Командир на Втора българска армия през Балканските войни от 1912–1913 година. Ръководи обсадата и превземането на Одринската крепост (март 1913).\n\nНикола Иванов е роден на 2 март 1861 г. в Калофер. Учи в Априловската гимназия '''
options_control = [
         "Лигата на нациите",
         "Организацията на обединените нации",
         "Международната агенция за атомна енергия",
         "Международната организация на труда"
      ]


        
predict(model, tokenizer, wiki_control, question_control, options_control, use_gpu=True)

array([[-1.49886918, -1.16217601, -1.42287099, -1.31306469]])

In [8]:
es = Elasticsearch()
name = "bgwiki_paragraph"

In [9]:
quiz = BGQuiz("data/bg_rc-v1.0.json")

In [10]:
!polyglot download embeddings2.bg
!polyglot download embeddings2.sr
!polyglot download embeddings2.ru

!polyglot download ner2.bg
!polyglot download ner2.sr
!polyglot download ner2.ru

!polyglot download pos2.bg


[polyglot_data] Downloading package embeddings2.bg to
[polyglot_data]     /home/momchil/polyglot_data...
[polyglot_data]   Package embeddings2.bg is already up-to-date!
[polyglot_data] Downloading package embeddings2.sr to
[polyglot_data]     /home/momchil/polyglot_data...
[polyglot_data]   Package embeddings2.sr is already up-to-date!
[polyglot_data] Downloading package embeddings2.ru to
[polyglot_data]     /home/momchil/polyglot_data...
[polyglot_data] Downloading package ner2.bg to
[polyglot_data]     /home/momchil/polyglot_data...
[polyglot_data]   Package ner2.bg is already up-to-date!
[polyglot_data] Downloading package ner2.sr to
[polyglot_data]     /home/momchil/polyglot_data...
[polyglot_data]   Package ner2.sr is already up-to-date!
[polyglot_data] Downloading package ner2.ru to
[polyglot_data]     /home/momchil/polyglot_data...
[polyglot_data]   Package ner2.ru is already up-to-date!
[polyglot_data] Downloading package pos2.bg to
[polyglot_data]     /home/momchil/polyglot_da

In [ ]:
from qa.wiki_abstracts import get_wiki_abstracts


query_fields = [
#     ['title.bulgarian', 'passage.bulgarian'],
#     ['title.bulgarian', 'passage.ngram'],
#     ['title.bulgarian', 'passage.ngram', 'passage', 'passage.bulgarian'],
#     ['passage.ngram', 'passage', 'passage.bulgarian^2'],
    ['title.bulgarian^2', 'passage.ngram', 'passage', 'passage.bulgarian^2'],
]

for query_field in query_fields:
    #, 'bgwiki_window'
    for name in ['bgwiki_paragraph']:
        for cr in [2]:
            start = time.time()
            
            setup = dict()
            setup['results_count'] = cr
            setup['query_field'] = query_field.copy()
            setup['index'] = name
            setup['model'] = model_dir

            print(json.dumps(setup, indent=2))
            print()

            preds = {}

            for i, q in enumerate(quiz.iterator()):
                question = q.question
                answers = q.answers

                try:
                    print(str(i + 1) + './' + str(quiz.size()), end='...')
                    print('QUESTION IS')
                    print(question)
                    
                    question_abstracts = get_wiki_abstracts([question])
                    question_abstracts = question_abstracts[0] if len(question_abstracts) > 0 else ''
                    answers_abstracts = get_wiki_abstracts(answers)
                    
                    passages = set()
                    assert len(answers) == len(answers_abstracts)
                    for i, a in enumerate(answers):
                        passages.update([
                            ' '.join((x['source']['passage'], answers_abstracts[i], question_abstracts))
                                     for x in query_es(
                                es, question + ' ' + a, setup['index'],
                                setup['results_count'], setup['query_field'])
                        ])
                    
                    # Adding wikipedia passages
                    
#                     passages = list(map(lambda x: x[0] + ' ' + x[1],
#                                         zip(passages, [question_abstracts + ' ' + a for a in answers_abstracts])))

                    ans_cnt = len(answers)
                    if ans_cnt < 4:
                        answers.append(' ')

                    votes = np.zeros(4, dtype=np.int32)
                    sc = np.ones((1, 4), dtype=np.float32)
                    for p in passages:
                        scores = predict(
                            model, tokenizer, p, question, answers, use_gpu=True)
                        if ans_cnt < 4:
                            scores[0, 3] = -np.inf
                        sc += softmax(scores)

                        votes[scores.argmax()] += 1

    #                 print(str(i + 1) + './' + str(len(quiz.data_gen)), question)
    #                 print(answers, q['correct'])
    #                 print(answers[sc.argmax()], votes)

                    preds[q.id] = answers[sc.argmax()]
                    print()
                except KeyboardInterrupt:
                    raise

            print()
            dirname = str(int(datetime.now().timestamp()))
            path = 'experiments/{}'.format(dirname)
           
            try:
                if not os.path.exists(path):
                    os.makedirs(path)
            except OSError:
                print("Creation of the directory %s failed" % path)
            else:
                print("Successfully created the directory %s " % path)

            with open(path + '/preds.json', 'w') as o:
                json.dump(preds, o, ensure_ascii=False)

            with open(path + '/setup.json', 'w') as o:
                json.dump(setup, o, ensure_ascii=False, indent=2)
                
            

            print(path)
            !python evaluation/evaluate.py data/bg_rc-v1.0.json $path/preds.json  --out-file $path/eval.json
            
            end = time.time()
            print("Done in {} seconds", int(end - start))

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


{
  "results_count": 2,
  "query_field": [
    "title.bulgarian^2",
    "passage.ngram",
    "passage",
    "passage.bulgarian^2"
  ],
  "index": "bgwiki_paragraph",
  "model": "models/RACE_BERT3"
}

1./2633...QUESTION IS
Самостоятелно съществуващи живи системи са:
Entity found системи
Entity found вирусите
Entity found тъканите
Entity found митохондриите
Entity found организми


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.



2./2633...QUESTION IS
Всички организми, населяващи една и съща територия и свързани помежду си с разнообразни отношения, образуват:
Entity found организми
Entity found територия
Entity found помежду
Entity found отношения


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


...

Entity found видът
Entity found биосферата

187./2633...QUESTION IS
Към групата на биотичните екологични фактори се отнася:
Entity found групата
Entity found фактори
Entity found въздействието
Entity found паразит
Entity found гостоприемник


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


Another language found
Entity found температурата
Entity found замърсяването
Entity found почвата
Entity found изригване
Entity found вулкан


Detector is not able to detect the language reliably.



188./2633...QUESTION IS
Популацията е съвкупност от:
Entity found Популацията
Entity found съвкупност
Entity found индивиди
Entity found видове
Entity found видове
Entity found видове
Entity found индивиди
Entity found вид

189./2633...QUESTION IS
Образуването на семейни групи (стада) при слоновете и маймуните е пример за:
Entity found Образуването
Entity found групи
Entity found стада


In [9]:
dirname = str(int(datetime.now().timestamp()))
path = 'experiments/{}'.format(dirname)

try:
    if not os.path.exists(path):
        os.makedirs(path)
except OSError:
    print("Creation of the directory %s failed" % path)
else:
    print("Successfully created the directory %s " % path)

with open(path + '/preds.json', 'w') as o:
    json.dump(preds, o, ensure_ascii=False)

with open(path + '/setup.json', 'w') as o:
    json.dump(setup, o, ensure_ascii=False, indent=2)



print(path)
!python evaluation/evaluate.py data/bg_rc-v1.0.json $path/preds.json  --out-file $path/eval.json

end = time.time()
print("Done in {} seconds", int(end - start))

Successfully created the directory experiments/1560647798 
experiments/1560647798
Done in {} seconds 13753


In [ ]:
from polyglot.text import Text


t = Text("Редуцентите са организми, които се хранят:", hint_language_code='bg')

In [ ]:
t.pos_tags

In [ ]:
list(filter(lambda x: x[1] == 'NOUN', t.pos_tags))